In [1]:
import numpy as np
import PyPDF2
import docx2txt
import sys

In [2]:
import matplotlib.pyplot as plt
%matplotlib inline 

In [3]:
import networkx as nx

In [4]:
from nltk.tokenize.punkt import PunktSentenceTokenizer

In [5]:
from sklearn.feature_extraction.text import TfidfTransformer, CountVectorizer

In [6]:
def readDoc():
    name = input('Please input a file name: ') 
    print('You have asked for the document {}'.format(name))

    if name.lower().endswith('.txt'):
        choice = 1
    elif name.lower().endswith('.pdf'):
        choice = 2
    else:
        choice = 3
    print(choice)
   
    if choice == 1:
        f = open(name, 'r')
        document = f.read()
        f.close()
            
    elif choice == 2:
        pdfFileObj = open(name, 'rb')
        pdfReader = PyPDF2.PdfFileReader(pdfFileObj)
        pageObj = pdfReader.getPage(0)
        document = pageObj.extractText()
        pdfFileObj.close()
    
    else:
        print('Failed to load a valid file')
        print('Returning an empty string')
        document = ''
    
    print(type(document))
    return document

In [7]:
def tokenize(document):
    doc_tokenizer = PunktSentenceTokenizer()

    sentences_list = doc_tokenizer.tokenize(document)
    return sentences_list

In [8]:
document = readDoc()
print('The length of the file is:', end=' ')
print(len(document))

Please input a file name: story.pdf
You have asked for the document story.pdf
2
<class 'str'>
The length of the file is: 78


In [9]:
sentences_list = tokenize(document)
print('The size of the list in Bytes is: {}'.format(sys.getsizeof(sentences_list)))
print('The size of the item 0 in Bytes is: {}'.format(sys.getsizeof(sentences_list[0])))

The size of the list in Bytes is: 64


IndexError: list index out of range

In [10]:
print(type(sentences_list))

<class 'list'>


In [11]:
print('The size of the list "sentences" is: {}'.format(len(sentences_list)))

The size of the list "sentences" is: 0


In [12]:
for i in sentences_list:
    print(i)

In [13]:
cv = CountVectorizer()
cv_matrix = cv.fit_transform(sentences_list)

ValueError: empty vocabulary; perhaps the documents only contain stop words

In [14]:
cv_demo = CountVectorizer()
text_demo = ["Goutham is good, you are bad", "I am not bad"] 
res_demo = cv_demo.fit_transform(text_demo)
print('Result demo array is {}'.format(res_demo.toarray()))
print('Feature list: {}'.format(cv_demo.get_feature_names()))

Result demo array is [[0 1 1 1 1 1 0 1]
 [1 0 1 0 0 0 1 0]]
Feature list: ['am', 'are', 'bad', 'good', 'goutham', 'is', 'not', 'you']


In [15]:
print('The data type of bow matrix {}'.format(type(cv_matrix)))
print('Shape of the matrix {}'.format(cv_matrix.get_shape))
print('Size of the matrix is: {}'.format(sys.getsizeof(cv_matrix)))
print(cv.get_feature_names())
print(cv_matrix.toarray())

NameError: name 'cv_matrix' is not defined

In [16]:
normal_matrix = TfidfTransformer().fit_transform(cv_matrix)
print(normal_matrix.toarray())

NameError: name 'cv_matrix' is not defined

In [17]:
print(normal_matrix.T.toarray)
res_graph = normal_matrix * normal_matrix.T
# plt.spy(res_graph)

NameError: name 'normal_matrix' is not defined

In [18]:
nx_graph = nx.from_scipy_sparse_matrix(res_graph)
nx.draw_circular(nx_graph)
print('Number of edges {}'.format(nx_graph.number_of_edges()))
print('Number of vertices {}'.format(nx_graph.number_of_nodes()))
plt.show()
print('The memory used by the graph in Bytes is: {}'.format(sys.getsizeof(nx_graph)))

NameError: name 'res_graph' is not defined

In [19]:
ranks = nx.pagerank(nx_graph)
print(type(ranks))
print('The size used by the dictionary in Bytes is: {}'.format(sys.getsizeof(ranks)))
for i in ranks:
    print(i, ranks[i])


NameError: name 'nx_graph' is not defined

In [20]:
sentence_array = sorted(((ranks[i], s) for i, s in enumerate(sentences_list)), reverse=True)
sentence_array = np.asarray(sentence_array)

In [21]:
rank_max = float(sentence_array[0][0])
rank_min = float(sentence_array[len(sentence_array) - 1][0])

IndexError: index 0 is out of bounds for axis 0 with size 0

In [22]:
print(rank_max)
print(rank_min)

NameError: name 'rank_max' is not defined

In [23]:
temp_array = []
flag = 0
if rank_max - rank_min == 0:
    temp_array.append(0)
    flag = 1
if flag != 1:
    for i in range(0, len(sentence_array)):
        temp_array.append((float(sentence_array[i][0]) - rank_min) / (rank_max - rank_min))

print(len(temp_array))

NameError: name 'rank_max' is not defined

In [24]:
threshold = (sum(temp_array) / len(temp_array)) + 0.2

ZeroDivisionError: division by zero

In [25]:
sentence_list = []
if len(temp_array) > 1:
    for i in range(0, len(temp_array)):
        if temp_array[i] > threshold:
                sentence_list.append(sentence_array[i][1])
else:
    sentence_list.append(sentence_array[0][1])

IndexError: index 0 is out of bounds for axis 0 with size 0

In [26]:
model = sentence_list

In [27]:
summary = " ".join(str(x) for x in sentence_list)
print(summary)
f = open('final3.txt', 'a+')
f.write('\n')
f.write(summary)
f.close

<function TextIOWrapper.close()>

In [28]:
for lines in sentence_list:
    print(lines)

In [29]:
# -*- coding: utf-8 -*-
from __future__ import print_function
from pythonrouge.pythonrouge import Pythonrouge
from pprint import pprint

if __name__ == '__main__':
    summary = './sample/summary/'
    reference = './sample/reference/'
    print('evaluate sumamry & reference in these dirs')
    print('summary:\t{}\nreference:\t{}'.format(summary, reference))
    rouge = Pythonrouge(summary_file_exist=True,
                        peer_path=summary, model_path=reference,
                        n_gram=2, ROUGE_SU4=True, ROUGE_L=False,
                        recall_only=False, f_measure_only=False,
                        stemming=True, stopwords=True,
                        word_level=True, length_limit=True, length=50,
                        use_cf=True, cf=95, scoring_formula='average',
                        resampling=True, samples=1000, favor=True, p=0.5)
    score = rouge.calc_score()
    print('ROUGE-N(1-2) & SU4 recall & f-measure with confidence interval')
    pprint(score)
    print('Evaluate ROUGE based on sentecnce lists')
    
    summary = [["An officer of the Central Reserve Police Force (CRPF) officer was killed in a gunfight",
                "with the Maoists in a dense forest in the insurgency-hit Sukma district of Chhattisgarh",
                "The gunfight took place last night in Kistaram police station area."],
               ["when a team of CRPF's 212th battalion was out on a search-cum-area domination operation",
                "Deputy Inspector General of Police",
                "Police (South Bastar Range) Sundarraj P told PTI"]]
    reference = [
                 [["An officer of the Central Reserve Police Force (CRPF) officer was killed in a gunfight "],
                  ["with the Maoists in a dense forest in the insurgency-hit Sukma district of Chhattisgarh"],
                  ["The gunfight took place last night in Kistaram police station area when a team of CRPF's",
                  "12th battalion was out on a search-cum-area domination operation,"]],
                 [["Deputy Inspector General of Police (South Bastar Range)",
                   " Sundarraj P told PTI."],
                  ["The deceased was a native of Madhya Pradesh.",
                   "Security forces have launched a combing operation in the area",
                   "ocated around 500 km away from the state capital Raipur"],
                  ]
                  ]
    rouge = Pythonrouge(summary_file_exist=False,
                        summary=summary, reference=reference,
                        n_gram=2, ROUGE_SU4=True, ROUGE_L=False,
                        recall_only=True, stemming=True, stopwords=True,
                        word_level=True, length_limit=True, length=50,
                        use_cf=True, cf=95, scoring_formula='average',
                        resampling=True, samples=1000, favor=True, p=0.5)
    score = rouge.calc_score()
    print('ROUGE-N(1-2) & SU4 recall only with confidence interval')
    pprint(score)

evaluate sumamry & reference in these dirs
summary:	./sample/summary/
reference:	./sample/reference/
ROUGE-N(1-2) & SU4 recall & f-measure with confidence interval
{}
Evaluate ROUGE based on sentecnce lists
ROUGE-N(1-2) & SU4 recall only with confidence interval
{'ROUGE-1': 0.61313,
 'ROUGE-1-cf95': (0.46154, 0.76471),
 'ROUGE-2': 0.54234,
 'ROUGE-2-cf95': (0.375, 0.70968),
 'ROUGE-SU4': 0.50672,
 'ROUGE-SU4-cf95': (0.34677, 0.66667)}
